In [20]:
# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp

from scipy.optimize import differential_evolution, minimize
from scipy.stats.qmc import Halton

# General imports
import os
import json
import numpy as np
from datetime import datetime, timedelta

from qiskit.quantum_info import SparsePauliOp

# custom module
from susy_qm import calculate_Hamiltonian

from qiskit_aer import Aer
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer.noise import NoiseModel

In [2]:
service = QiskitRuntimeService()
backend = service.backend('ibm_brisbane')

In [3]:
#from qiskit_ibm_runtime.fake_provider import FakeAlmadenV2
#backend = FakeAlmadenV2()
noise_model = NoiseModel.from_backend(backend)

In [26]:
potential = 'AHO'
cutoff = 8
shots = 10000

In [27]:
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])[:4]
min_eigenvalue = min(eigenvalues.real)

num_qubits = int(1 + np.log2(cutoff))

In [28]:
dev = qml.device('qiskit.aer', wires=num_qubits, noise_model=noise_model, shots=shots, seed=42)

@qml.qnode(dev)
def cost_function(params):

    basis_state = [1] + [0]*(num_qubits-1)
    qml.BasisState(basis_state, wires=range(num_qubits))

    qml.RY(params[0], wires=1)
    qml.RY(params[1], wires=2)
    #qml.RY(params[2], wires=2)
    #qml.RY(params[1], wires=[num_qubits-1])
    qml.CRY(params[2], wires=[2, 1])
    #qml.RY(params[3], wires=[num_qubits-2])
    #qml.RY(params[4], wires=[num_qubits-1])
    
    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))
    

In [29]:
# VQE
vqe_start = datetime.now()

#variables
num_vqe_runs = 1
max_iter = 100
strategy = "randtobest1bin"
tol = 1e-3
abs_tol = 1e-3
popsize = 5

# Generate Halton sequence
num_dimensions = 3
num_samples = popsize
halton_sampler = Halton(d=num_dimensions)
halton_samples = halton_sampler.random(n=num_samples)
scaled_samples = 2 * np.pi * halton_samples

#data arrays
energies = []
x_values = []
success = []
run_times = []
num_iters = []
num_evaluations = []

#Optimizer
bounds = [(0, 2 * np.pi) for _ in range(num_dimensions)]
x0 = np.random.random(size=num_dimensions)*2*np.pi

for i in range(num_vqe_runs):

    run_start = datetime.now()

    if i % 10 == 0:
        print(f"Run: {i}")

    '''
    # Differential Evolution optimization
    res = differential_evolution(cost_function,
                                bounds,
                                maxiter=max_iter,
                                tol=tol,
                                atol=abs_tol,
                                strategy=strategy,
                                popsize=popsize,
                                init=scaled_samples
                                )
    '''
    res = minimize(
            cost_function,
            x0,
            method= "COBYLA",
            options= {'maxiter':max_iter, 'tol':tol}
        )

    if res.success == False:
        print("Not converged")

    energies.append(res.fun)
    x_values.append(res.x)
    success.append(res.success)
    num_iters.append(res.nfev)
    #num_iters.append(res.nit)
    #num_evaluations.append(res.nfev)

    run_end = datetime.now()
    run_time = run_end - run_start
    run_times.append(run_time)

total_run_time = sum(run_times, timedelta())
vqe_end = datetime.now()
vqe_time = vqe_end - vqe_start

Run: 0


In [31]:
print(min_eigenvalue, energies)

0.03201011000923872 [np.float64(39.86011041287141)]


In [19]:
# Save run
run = {
    #"starttime": starttime,
    "potential": potential,
    "cutoff": cutoff,
    "exact_eigenvalues": [x.real.tolist() for x in eigenvalues],
    "ansatz": "StronglyEntanglingLayers-1layer",
    "num_VQE": num_vqe_runs,
    "shots": shots,
    "Optimizer": {
        "name": "differential_evolution",
        "bounds": "[(0, 2 * np.pi) for _ in range(np.prod(params_shape))]",
        "maxiter": max_iter,
        "tolerance": tol,
        "abs_tolerance": abs_tol,
        "strategy": strategy,
        "popsize": popsize,
        'init': 'scaled_samples',
    },
    "results": energies,
    "params": [x.tolist() for x in x_values],
    "num_iters": num_iters,
    "num_evaluations": num_evaluations,
    "success": np.array(success, dtype=bool).tolist(),
    "run_times": [str(x) for x in run_times],
    "parallel_run_time": str(vqe_time),
    "total_VQE_time": str(total_run_time)
}

run

{'potential': 'AHO',
 'cutoff': 16,
 'exact_eigenvalues': [-0.001166975680574785,
  1.6774941961816563,
  1.6863812488131036,
  4.364321396210466],
 'ansatz': 'StronglyEntanglingLayers-1layer',
 'num_VQE': 1,
 'shots': 2,
 'Optimizer': {'name': 'differential_evolution',
  'bounds': '[(0, 2 * np.pi) for _ in range(np.prod(params_shape))]',
  'maxiter': 100,
  'tolerance': 0.001,
  'abs_tolerance': 0.001,
  'strategy': 'randtobest1bin',
  'popsize': 5,
  'init': 'scaled_samples'},
 'results': [np.float64(-0.0011669756806017494)],
 'params': [[1.3439955151618823, 1.6870449623891193, 1.55988406008951]],
 'num_iters': [100],
 'num_evaluations': [605],
 'success': [False],
 'run_times': ['0:07:23.761694'],
 'parallel_run_time': '0:07:23.762647',
 'total_VQE_time': '0:07:23.761694'}

In [22]:
# Save the variable to a JSON file
base_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\VQE\Differential Evolution\Noise Simulator\NoiseSimFiles\\{}\\".format(potential)
path = base_path + "{}_{}.json".format(potential, cutoff)
with open(path, 'w') as json_file:
    json.dump(run, json_file, indent=4)